# LLM의 출력 품질 향상 방법

## 사전 준비 : 가상 환경 구축, 패키지 설치

In [37]:
#1

!pip uninstall -y google-generativeai
!pip install langchain==0.3.20 langchain_core==0.3.49 langchain_google_genai==2.1.1 httpx==0.28.1 requests==2.32.3

import os

#환경 변수 및 패키지 준비
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
#os.environ["GOOGLE_API_KEY"] = "****"


In [43]:
#2

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate,  HumanMessagePromptTemplate
from langchain_core.messages import  SystemMessage

system_prompt="""
답변할 때 아래와 같이 구조화하여 출력하세요.
- reasoning: 사용자의 질문을 단계별로 고려하고, 최종 답변을 추론하고, 모든 생각을 이 필드에 출력하세요.
- conclusion: 최종 답변만 출력하세요.
"""

# prompt 템플릿 정의
message = [
        SystemMessage(content= system_prompt),
        HumanMessagePromptTemplate.from_template(
            [
                {
                    "type": "text",
                    "text": "{question}"
                },
            ]
        )
    ]


prompt = ChatPromptTemplate.from_messages(message)


# gemini 모델 정의
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
)


In [58]:
#3

from pydantic import BaseModel, Field

# 출력 구조를 모델 클래스로 정의
class OutputModel(BaseModel):
    reasoning: str = Field(..., description='문제를 푸는 데 필요한 모든 사고 내용과, 최종적인 결과를 출력한다.')
    conclusion: str = Field(..., description='지금까지의 사고 결과로부터 최종 결론만을 출력한다.')

chain = prompt | model.with_structured_output(OutputModel)


question = """
공원과 학교 사이의 거리는 1200m입니다.
A군이 공원에서, B군은 학교에서 마주보고 동시에 출발하여 8분만에 만났습니다.
B군이 학교에서 나온지 5분 뒤에 A군이 쫓아거가 시작해 A군이 출발한지 10분만에 따라잡혔다.
A군의 속도는 얼마나 빨리 갔나요?
"""

result = chain.invoke({"question": question})

print("########## 사고 과정 ##########")
print(result.reasoning)
print("########## 결론 ##########")
print(result.conclusion)


########## 사고 과정 ##########
1. A와 B가 8분 동안 이동한 거리의 합은 1200m입니다. 즉, 8A + 8B = 1200입니다.
2. A가 B를 따라잡는 데 걸린 시간은 10분이고, B는 A가 출발하기 전 5분 동안 먼저 이동했으므로, A가 이동한 거리는 10A이고 B가 이동한 거리는 15B입니다. 즉, 10A = 15B입니다.
3. 위의 두 식을 연립하여 풀면 A = 100, B = 50입니다.
따라서 A의 속도는 분속 100m, B의 속도는 분속 50m 입니다.

########## 결론 ##########
A의 속도는 분속 100m, B의 속도는 분속 50m 입니다.


In [57]:
# 4
message = [
        SystemMessage(content= "답만 간결하게 출력해 주세요. 답 이외에는 출력을 금지합니다."),
        HumanMessagePromptTemplate.from_template(
            [
                {
                    "type": "text",
                    "text": "{question}"
                },
            ]
        )
    ]

prompt = ChatPromptTemplate.from_messages(message)


# gemini 모델 정의
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
)

chain = prompt | model

question = """
공원과 학교 사이의 거리는 1200m입니다.
A군이 공원에서, B군은 학교에서 마주보고 동시에 출발하자 8분만에 만났습니다.
B군이 학교에서 나온 5분 뒤 A군이 쫓아가지 시작해 A군이 출발한지 10분만에 따라잡혔다.
A군의 속도는 얼마나 빨리 갔나요?
"""

print("########## model 결과 ##########")
print(chain.invoke({"question": question}).content)



########## model 결과 ##########
분속 80m
